In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import BertNormalizer

In [2]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [3]:
# order of special tokens is important
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [5]:
tokenizer.pre_tokenizer = Whitespace()
tokenizer.normalizer = BertNormalizer()

In [6]:
from datasets import load_dataset

In [8]:
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split=["train", "test"])

Dataset wikitext downloaded and prepared to /Users/sidbaskaran/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 11.53it/s]


In [11]:
files = [f"../../data/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [12]:
tokenizer.save("../../data/tokenizer-wiki.json")

In [17]:
# testing the tokenizer
out = tokenizer.encode("Always trying to make the most of bruh moments in life.")

In [18]:
out.tokens

['Al',
 'ways',
 'trying',
 'to',
 'make',
 'the',
 'most',
 'of',
 'bru',
 'h',
 'moments',
 'in',
 'life',
 '.']

In [19]:
out.ids

[5345,
 6428,
 9210,
 5030,
 6052,
 5015,
 5340,
 5027,
 23616,
 76,
 12367,
 5012,
 5836,
 18]

In [20]:
from tokenizers.processors import TemplateProcessing
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)